# Imports 

In [8]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
from torchinfo import summary
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import warnings
from matplotlib.patches import Rectangle
import tkinter as tk
from tkinter import filedialog, messagebox
import numpy as np
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import random

Data splitting (Train / Validation / Test)

In [ ]:
# --- Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# NOTE: The data path has been set to your specified Windows location.
DATA_DIR = r"C:\Users\mirab\OneDrive\Documents\junior\basic of AI programming\archive.zip\plantvillage dataset\color" 

# Hyperparameters
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 20 # Start with a small number, increase for full training
SPLIT_RATIOS = {'train': 0.7, 'val': 0.2, 'test': 0.1}

Using device: cuda


Data Transforms

In [3]:
# Define data transforms with required augmentations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(), # Orientation Augmentation
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1), # Color Augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load the full dataset
try:
    full_dataset = datasets.ImageFolder(DATA_DIR)
except Exception as e:
    print(f"Error loading dataset from {DATA_DIR}. Please check the path and file structure.")
    print(f"Error details: {e}")
    # Exit or use dummy data if loading fails
    raise

class_names = full_dataset.classes
num_classes = len(class_names)

print(f"Total classes found: {num_classes}")
print(f"Class names: {class_names}")
print(f"Total images in dataset: {len(full_dataset)}")

# Get indices for the full dataset
all_indices = list(range(len(full_dataset)))

# Calculate split sizes
train_size = int(SPLIT_RATIOS['train'] * len(full_dataset))
val_size = int(SPLIT_RATIOS['val'] * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size # Remaining for test

# 1. Split Train/Temp (70% / 30%)
train_indices, temp_indices = train_test_split(
    all_indices, 
    train_size=train_size, 
    test_size=val_size + test_size, 
    stratify=full_dataset.targets, 
    random_state=42
)

# 2. Split Val/Test from the remaining (20% / 10%)
val_ratio = val_size / (val_size + test_size)

val_indices, test_indices = train_test_split(
    temp_indices, 
    train_size=val_ratio, 
    stratify=[full_dataset.targets[i] for i in temp_indices], 
    random_state=42
)

# Create Subsets with appropriate transforms
train_dataset = Subset(datasets.ImageFolder(DATA_DIR, transform=data_transforms['train']), train_indices)
val_dataset = Subset(datasets.ImageFolder(DATA_DIR, transform=data_transforms['val']), val_indices)
test_dataset = Subset(datasets.ImageFolder(DATA_DIR, transform=data_transforms['test']), test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0) # num_workers=0 for Windows compatibility
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print("\n--- Dataset Split Summary ---")
print(f"Train set size: {len(train_dataset)} ({SPLIT_RATIOS['train']*100}%)")
print(f"Validation set size: {len(val_dataset)} ({SPLIT_RATIOS['val']*100}%)")
print(f"Test set size: {len(test_dataset)} ({SPLIT_RATIOS['test']*100}%)")

Total classes found: 38
Class names: ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy', 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy', 'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight', 'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spot

Build Model

In [4]:
def get_resnet50_model(num_classes):
    # Load pre-trained ResNet50
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    
    # Freeze all the parameters in the network
    for param in model.parameters():
        param.requires_grad = False

    # Replace the final fully connected layer with a new one for our number of classes
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    
    # Unfreeze the final layer and the last few convolutional blocks for fine-tuning
    for param in model.fc.parameters():
        param.requires_grad = True
        
    # Unfreeze the parameters of the last convolutional block (layer4)
    for param in model.layer4.parameters():
        param.requires_grad = True
        
    return model

model = get_resnet50_model(num_classes).to(DEVICE)

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
# Use a lower learning rate for fine-tuning
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE) 
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1) # Decay LR every 5 epochs

print("Model setup complete. Ready for training.")

Model setup complete. Ready for training.


Accuracy Plot

In [ ]:
def plot_training_history(history):
    epochs = range(1, len(history['train_acc']) + 1)

    plt.figure(figsize=(8,5))
    plt.plot(epochs, history['train_acc'])
    plt.plot(epochs, history['val_acc'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'])
    plt.grid(True)
    plt.show()

Loss Plot

In [ ]:
def plot_loss_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(8,5))
    plt.plot(epochs, history['train_loss'])
    plt.plot(epochs, history['val_loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Validation'])
    plt.grid(True)
    plt.show()

Layers 

In [7]:
backbone = models.resnet50(pretrained=True)
for name, layer in backbone.named_children():
    print(name, layer)

c:\Users\pc\miniconda3\envs\ai4beg\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\pc\miniconda3\envs\ai4beg\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


conv1 Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
bn1 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
relu ReLU(inplace=True)
maxpool MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
layer1 Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05